# Introduction to using stdpopsim

## Workshop Outline
1. Basics of using Jupyter Notebooks
2. Overview of stdpopsim
3. How to use the Python API
4. How to use the command line interface
5. Example analysis
6. How to ask for help
7. Some examples of what stdpopsim cannot currently do
8. Teaser of how to contribute
9. Using stdpopsim after the workshop  
10. How to Navigate the stdpopsim library catalog

----------
## 1. Basics of using Jupyter Notebooks

Jupyter Notebooks have cells where you can write in Markdown and run code.  
To execute a cell, click the run button or press shift enter.

In [ ]:
print('Try writing some Python here')

In [ ]:
%%bash
echo 'We can also use Bash magic. Try writing some Bash here'

---------
## 2. Overview of stdpopsim

### What is stdpopsim?
- Library of previously published population genetic models that can be used to simulate data
- Includes simple & complex demographic models, species specific recombination maps, and our newest feature-- selection
- Models have undergone rigorous quality control to ensure what we implement matches the original publication

### Why is stdpopsim useful?
- Increase reproducibility in population genetics modeling
- Less work for simulating data to test new inference methods
- Facilitate comparisons among inference methods

### Phase 1
Adrion et al. (2020). _A community-maintained standard library of population genetic models_. eLife. https://doi.org/10.7554/eLife.54967

- Focused just on demographic modeling
- Used msprime as simulation engine
- Realistic genetic maps for each species

### Phase 2
Lauterbur et al. (2023) _Expanding the stdpopsim species catalog, and lessons learned for realistic genome simulations_ eLife. https://doi.org/10.7554/eLife.84874.1

- Expanded the catalog to 21 species!
- SLiM now fully functional as backend simulation engine
- Functions for modeling selection introduced

### Phase 3
New manuscript coming this year!

- Implemented models of selection in numerous genomes
- Published DFEs included
- Species-specific annotations (CDS, exons) included

### How to use stdpopsim
- Python API
  - more flexible
  - access to tskit functionality
  - faster for more replicates
- command line interface
  - one-liner
  - nice for people familiar with command line
  - easy to put into a bigger workflow

---------------
## 3. How to use the Python API

### Import the stdpopsim package

In [ ]:
import stdpopsim

### Open the catalog page

Resources:
- [Catalog](https://stdpopsim.readthedocs.io/en/stable/catalog.html#)
- [Documentation](https://stdpopsim.readthedocs.io/en/stable/api.html)
- [Tutorials](https://stdpopsim.readthedocs.io/en/stable/tutorial.html#running-stdpopsim-with-the-python-interface-api)

### Which species?

Let's simulate humans.

_Use the [catalog](https://stdpopsim.readthedocs.io/en/stable/catalog.html#) to find available species_

https://stdpopsim.readthedocs.io/en/stable/api.html#stdpopsim.get_species

In [ ]:
species = stdpopsim.get_species("HomSap")

### Which chromosome?

We'll simulate a region that is 10% of the length of chromosome 22.

Find what chromosomes are available in the [catalog](https://stdpopsim.readthedocs.io/en/stable/catalog.html#)

https://stdpopsim.readthedocs.io/en/stable/api.html#stdpopsim.Species.get_contig

In [ ]:
contig = species.get_contig("chr22", length_multiplier=0.1)

Using a *real* genetic map: (cannot be used in conjunction with `length_multiplier`)

In [ ]:
#contig = species.get_contig("chr22", genetic_map="HapMapII_GRCh38")

Using an alternative mutation or recombination rate:

In [ ]:
# other_contig = stdpopsim.Contig(
#     mutation_rate=contig.mutation_rate*0.5,
#     recombination_map=contig.recombination_map
# ) 



### Which demographic model?
https://stdpopsim.readthedocs.io/en/stable/api.html#stdpopsim.DemographicModel

Use the [catalog](https://stdpopsim.readthedocs.io/en/stable/catalog.html#) to find available demographic models.
You can also see them in the `species` object you just created:

In [ ]:
for model in species.demographic_models:
    print(model.id)

Then set your demographic model.
https://stdpopsim.readthedocs.io/en/stable/api.html#stdpopsim.Species.get_demographic_model

In [ ]:
model = species.get_demographic_model('OutOfAfrica_3G09')

You may want to verify that the simulated populations are correct.

In [ ]:
print([pop.name for pop in model.populations])

### How many samples?

In [ ]:
samples = {"YRI": 5, "CHB": 5, "CEU": 0}

### Which simulator?
https://stdpopsim.readthedocs.io/en/stable/api.html#simulation-engines

In [ ]:
engine = stdpopsim.get_engine('slim')

We're now ready to simulate:
https://stdpopsim.readthedocs.io/en/stable/api.html#stdpopsim.Engine.simulate

In [ ]:
%%time
ts = engine.simulate(model, contig, samples, slim_scaling_factor=100, seed=1)

## Output
### to a tree sequence file

In [ ]:
ts.dump("OutOfAfrica_3G09_API.ts")

### to a vcf file

In [ ]:
with open("OutOfAfrica_3G09_API.vcf", "w") as vcf_file:
    ts.write_vcf(vcf_file, contig_id='22')

Let's take a look at our vcf:

In [ ]:
%%bash
ls

In [ ]:
%%bash
head OutOfAfrica_3G09_API.vcf

### All together now!

In [ ]:
import stdpopsim
species = stdpopsim.get_species("HomSap")
model = species.get_demographic_model('OutOfAfrica_3G09')
samples = {"YRI": 5, "CHB": 5, "CEU": 0}
engine = stdpopsim.get_engine('msprime')
contig = species.get_contig("chr22", length_multiplier=0.1)
ts = engine.simulate(model, contig, samples)
ts.dump("OutOfAfrica_3G09_API.trees")
with open("OutOfAfrica_3G09_API.vcf", "w") as vcf_file:
    ts.write_vcf(vcf_file, contig_id='22')

### Exercise: _Try to simulate the Arabidopsis South Middle Atlas piecewise constant size model!_

In [ ]:
species = stdpopsim.get_species("") #fill me!
model = species.get_demographic_model('') #fill me!
samples =  #fill me!
engine = stdpopsim.get_engine('msprime')
contig = species.get_contig("", length_multiplier=0.1) #fill me!
ts = engine.simulate(model, contig, samples)

## Questions?
--------------------

## Adding selection
To add selection into our simulations we need to add two components
1. the regions of the simulated chromosome that we wish to be under selection
2. the distribution of fitness effects (DFE) for new mutations in those regions

### Choose a DFE
DFEs appear in the catalog under each species. Here is the [human list](https://popsim-consortium.github.io/stdpopsim-docs/stable/catalog.html#sec_catalog_HomSap_dfe) 

a DFE is itself a python class that we can work with and compose onto chromosomes,
we have chosen to implement published ones in the catalog (e.g. the following example from Kim et al. 2017)


In [ ]:
species = stdpopsim.get_species("HomSap")
dfe = species.get_dfe("Gamma_K17")
print(dfe)

We will set up a Contig, specifying the set of `intervals` that the chosen DFE will apply to:



In [ ]:
import numpy as np
contig = species.get_contig("chr6", left=0, right=100000)
# make the DFE apply to the entire contig 
contig.add_dfe(intervals=np.array([[0, int(contig.length)]]), DFE=dfe)
# choose a demographic model if desired
model = species.get_demographic_model("OutOfAfrica_3G09")
samples = {"YRI": 50, "CEU": 50, "CHB": 50}

And now we will simulate. Simulations with selection are done forward in time using the SLiM simulation engine as a backend, so there are additional options we can specify about how to run our simulation 

In [ ]:
%%time
engine = stdpopsim.get_engine("slim")
ts = engine.simulate(
    model,
    contig,
    samples,
    seed=123,
    slim_scaling_factor=10, #this is a scalar for efficiency but leads to potential inacurracies
    slim_burn_in=10,
)

### Adding selection to exons
The catalog also has a certain number of annotations available, obtained from Ensembl. For instance, for humans we have:

In [ ]:
for a in species.annotations:
    print(f"{a.id}: {a.description}")

# ensembl_havana_104_exons: Ensembl Havana exon annotations on GRCh38
# ensembl_havana_104_CDS: Ensembl Havana CDS annotations on GRCh38

We’ll simulate with the `HomSap/Gamma_K17` DFE, applied to all exons in the region of chromosome 6 that spans from 10 to 30 Mb. Parts of this chromosomal region that aren’t exons will have only neutral mutations. To do so, we extract the intervals from the Annotation object and use this in `Contig.add_dfe()`:

In [ ]:
%%time
species = stdpopsim.get_species("HomSap")
dfe = species.get_dfe("Gamma_K17")
contig = species.get_contig("chr6", left=10e6, right=30e6)
model = species.get_demographic_model("OutOfAfrica_3G09")
samples = {"YRI": 50, "CEU": 50, "CHB": 50}

exons = species.get_annotations("ensembl_havana_104_exons")
exon_intervals = exons.get_chromosome_annotations("chr6")
contig.add_dfe(intervals=exon_intervals, DFE=dfe)

engine = stdpopsim.get_engine("slim")
ts = engine.simulate(
    model,
    contig,
    samples,
    seed=236,
    slim_scaling_factor=20,
    slim_burn_in=10,
)

This is a quick example because of the heavy scaling $Q=20$ that we are using but nonetheless there is lower diversity in exons than outside of them:


In [ ]:
breaks, labels = contig.dfe_breakpoints()

diffs = ts.diversity(windows=breaks, span_normalise=False)
pi = (
    np.sum(diffs[labels == 1]) / np.sum(np.diff(breaks)[labels == 1]),
    np.sum(diffs[labels == 0]) / np.sum(np.diff(breaks)[labels == 0]),
)

print(
    f"Mean sequence diversity in exons is {1000 * pi[0]:.3f} differences per Kb,\n"
    f"and outside of exons it is {1000 * pi[1]:.3f} differences per Kb."
)


### Exercise: _Try to simulate the Arabidopsis South Middle Atlas piecewise constant size model with selection on exons and a Gamma DFE!_

In [ ]:
species = stdpopsim.get_species("") #fill me!
model = species.get_demographic_model('') #fill me!
samples =  #fill me!


contig = species.get_contig("", length_multiplier=0.1) #fill me!
dfe = species.get_dfe("") #fill me!
exons = species.get_annotations("") #fill me!
exon_intervals = exons.get_chromosome_annotations("") #fill me!
contig.add_dfe(intervals=exon_intervals, DFE=dfe)

engine = stdpopsim.get_engine('') #fill me!
ts = engine.simulate(
    model,
    contig,
    samples,
    seed=236,
    slim_scaling_factor=20,
    slim_burn_in=10,
)

### Questions?

---------------
## 4. How to use the command line interface
Resources:
- [Documentation](https://stdpopsim.readthedocs.io/en/stable/cli_arguments.html)
- [Tutorials](https://stdpopsim.readthedocs.io/en/stable/tutorial.html#running-stdpopsim-with-the-command-line-interface-cli)

### Run stdpopsim with the help option

In [ ]:
%%bash
stdpopsim --help

`stdpopsim` uses a combination of [_positional arguments_](https://stdpopsim.readthedocs.io/en/stable/cli_arguments.html#Positional%20Arguments), which are required, and [_named arguments_](https://stdpopsim.readthedocs.io/en/stable/cli_arguments.html#Named%20Arguments), which are optional.

### Find your species and run stdpopsim with that species with the help option

In [ ]:
%%bash
stdpopsim HomSap --help

### Find your model and run stdpopsim with that model with the help option

In [ ]:
%%bash
stdpopsim HomSap \
        --help-models

## Run a simulation
- pick the number of samples for each population
- pick a chromosome
- do you want a fraction of the chromosome?
- decide on an output file

Let's do a dry run first by using `-D`.

In [ ]:
%%bash
stdpopsim HomSap YRI:10 CEU:10 CHB:10\
    --demographic-model OutOfAfrica_3G09 \
    --chromosome chr22 \
    --length-multiplier 0.1 \
    --output OutOfAfrica_3G09_CLI.ts \
    -D 

Does that look right? If so, let's do the simulation!

In [ ]:
%%bash
stdpopsim HomSap YRI:10 CEU:10 CHB:10\
         --demographic-model OutOfAfrica_3G09 \
         --chromosome chr22 \
         --length-multiplier 0.1 \
         --output OutOfAfrica_3G09_CLI.ts

In [ ]:
%%bash
tskit vcf OutOfAfrica_3G09_CLI.ts > OutOfAfrica_3G09_CLI.vcf

Let's take a look at our vcf.

In [ ]:
%%bash
ls

In [ ]:
%%bash
head OutOfAfrica_3G09_CLI.vcf

### _Try to simulate the Arabidopsis South Middle Atlas piecewise constant size model!_

Bonus: from the `--help` output, can you see how you might use one of the published genetic maps in this simulation? (perhaps the *SalomeAveraged_TAIR7* map?)

In [ ]:
%%bash
stdpopsim #fill me!

### Questions?

-------------
## 5. Example analysis
Let's suppose we wanted to check if a published model is a good approximation of our real data. To do this, we'll calculate a few population genetics statistics and see if the real data overlap our simulated data. (For this exercise we will assume some value for the stats from the real data).


### Our pretend real data
Let's say we have samples from three human populations, and they have a mean genetic diversity of: 0.00045, 0.00029, 0.00027.

In [ ]:
piA = 0.00045
piB = 0.00029
piC = 0.00027

### Simulate a model to compare to real data and calculate stats

We'll simulate the human Three population out-of-Africa _n_ times.  

_Find this model in the catalog._  
_How many populations are there?_  
_How many samples do we want to simulate?_  
_What chromosome do we want to simulate?_  
_Do we want to reduce the chromosome size?_

#### Set up the simulation

In [ ]:
import stdpopsim
species = stdpopsim.get_species("HomSap")
model = species.get_demographic_model('OutOfAfrica_3G09')
samples = {"YRI": 10, "CHB": 10, "CEU": 10}
engine = stdpopsim.get_engine('msprime')
contig = species.get_contig("chr8", left=1e6, right=5e6)


#### Simulation and calculate the stats in a for loop

Make a list of sample chromosomes (nodes) from each population

In [ ]:
def make_sample_list(ts):
    sample_list = []
    for pop in range(0, ts.num_populations):
        sample_list.append(ts.samples(pop).tolist())
    return sample_list

Run the simulations and calculate the summary statistics

In [ ]:
%%time
from progressbar import ProgressBar #This is just to get a progress bar
pbar = ProgressBar() #This is just to get a progress bar

n = 50
pi_list = []
for i in pbar(range(n)):
    ts = engine.simulate(model, contig, samples)
    sample_list = make_sample_list(ts)
    pi_list.append(ts.diversity(sample_sets=sample_list))
print('Done simulating {} replicates!'.format(n))

### Plot stats from simulated and "real" data

Convert simulated stats to a dataframe (only dataframes are easy to work with), then lets look at the first couple of lines using `head()`

In [ ]:
import pandas as pd
pi_df = pd.DataFrame(
    data=pi_list, columns=[pop.name for pop in model.populations])
pi_df.head()

In [ ]:
pi_df_melted = pd.melt(pi_df, var_name='Pops', value_name='Pi')

## Plot data

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
ax = sns.histplot(data=pi_df_melted, x="Pi", hue="Pops", bins=30)
ax.axvline(x=piA, color='blue')
ax.axvline(x=piB, color='orange')
ax.axvline(x=piC, color='green')
plt.xticks(rotation=45)

## A PCA example using scikit-allel
Quick look at using stdpopsim together with scikit-allel to examine PCA of genetic variation from simulations.

First we set the model:

In [ ]:
import allel
import stdpopsim

species = stdpopsim.get_species("HomSap")
model = species.get_demographic_model('OutOfAfrica_3G09')
samples = {"YRI": 50, "CHB": 50, "CEU": 50}
engine = stdpopsim.get_engine('msprime')
contig = species.get_contig("chr22", length_multiplier=0.1)

next we will simulate and use tools from scikit-allel to go from tree sequences to genotypes. we'll output the first few rows of the genotype array produced by allel

In [ ]:
ts = engine.simulate(model, contig, samples)
haps = allel.HaplotypeArray(ts.genotype_matrix())
gns = haps.to_genotypes(ploidy=2)
gns

now we're ready to do PCA. we'll do the decomposition and then plot, note that allel requires a flattened version of the genotype array above that codes individual genotypes as 0/1/2 according to the number of reference alleles

In [ ]:

def plot_pca_coords(coords, fitted, pc1, pc2, ax):
    x = coords[:, pc1]
    y = coords[:, pc2]
    ax.plot(x[0:25], y[0:25], marker='o', linestyle=' ', label='CEU', markersize=6)
    ax.plot(x[25:50], y[25:50], marker='o', linestyle=' ', label='CHB', markersize=6)
    ax.plot(x[50:75], y[50:75], marker='o', linestyle=' ', label='YRI', markersize=6)

    ax.set_xlabel('PC%s (%.1f%%)' % (pc1+1, fitted.explained_variance_ratio_[pc1]*100))
    ax.set_ylabel('PC%s (%.1f%%)' % (pc2+1, fitted.explained_variance_ratio_[pc2]*100))


fig, ax = plt.subplots(figsize=(6, 6))
sns.despine(ax=ax, offset=10)

coords, fitted = allel.pca(gns.to_n_ref())
plot_pca_coords(coords, fitted, 0, 1, ax)
ax.legend()



## Reduction in polymorphism due to selection
Let's simulate the same out-of-Africa model as above, add selection at exons, and compare polymorphism


In [ ]:
import stdpopsim
from progressbar import ProgressBar #This is just to get a progress bar

species = stdpopsim.get_species("HomSap")
model = species.get_demographic_model('OutOfAfrica_3G09')
samples = {"YRI": 10, "CHB": 10, "CEU": 10}
engine = stdpopsim.get_engine('msprime')
contig = species.get_contig("chr8", left=1e6, right=5e6)

exons = species.get_annotations("ensembl_havana_104_exons")
exon_intervals = exons.get_chromosome_annotations("chr8")
contig.add_dfe(intervals=exon_intervals, DFE=dfe)

engine = stdpopsim.get_engine("slim")

n = 50
pi_list = []
pbar = ProgressBar() #This is just to get a progress bar
for i in pbar(range(n)):
    ts = engine.simulate(model, contig, samples, slim_scaling_factor=20, slim_burn_in=10)
    sample_list = make_sample_list(ts)
    pi_list.append(ts.diversity(sample_sets=sample_list))
print('Done simulating {} replicates!'.format(n))



In [ ]:
pi_df_sel = pd.DataFrame(
    data=pi_list, columns=[pop.name for pop in model.populations])
pi_df_sel_melted = pd.melt(pi_df_sel, var_name='Pops', value_name='Pi')

import seaborn as sns
import matplotlib.pyplot as plt
ax = sns.histplot(data=pi_df_sel_melted, x="Pi", hue="Pops", bins=30)
ax.axvline(x=piA, color='blue')
ax.axvline(x=piB, color='orange')
ax.axvline(x=piC, color='green')
plt.xticks(rotation=45)

In [ ]:
pi_df_melted['selection'] = 'neutral'
pi_df_sel_melted['selection'] = 'Gamma DFE'                    
sns.boxplot(data=pi_df_melted, y='Pi', x='selection', hue='Pops')
sns.boxplot(data=pi_df_sel_melted, y='Pi', x='selection', hue='Pops', legend=False)


### Questions?

---------
## 6. How to ask for help
- Have you read the [documentation](https://stdpopsim.readthedocs.io/en/stable/index.html)?
- Search open and closed [GitHub issues](https://github.com/popsim-consortium/stdpopsim/issues?q=is%3Aissue)
- Write a new [GitHub issue](https://github.com/popsim-consortium/stdpopsim/issues/new/choose)
- Join the PopSim Slack workspace and post in the #newbie-help channel ([invite link here](https://join.slack.com/t/popsimgroup/shared_invite/zt-1ql1axuk6-ymqHmu9NQJ1PqfZKFC6SzQ))

---------
## 7. Some examples of what stdpopsim cannot currently do
- simulate species or demographic models that are not in the catalog 
    - if you want to do this, if it is a published model - submit it to stdpopsim, if it is not a published model, use a simulator (e.g. msprime, slim)
- simulate missing data and errors (on the horizon!)


----------
## 8. Teaser of [how to contribute](https://stdpopsim.readthedocs.io/en/stable/development.html#)

- Write [GitHub issues](https://github.com/popsim-consortium/stdpopsim/issues/new/choose)!
- [Add new species](https://stdpopsim.readthedocs.io/en/stable/development.html#adding-a-new-species)
- [Add new demographic models](https://stdpopsim.readthedocs.io/en/stable/development.html#adding-a-new-demographic-model)
- Help with Documentation and tutorials

-----------
## 9. Using stdpopsim on your own after the workshop
- Play in a Jupyter Notebook Binder
  - Using this one [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/popsim-consortium/workshops.git/main?filepath=intro_stdpopsim%2FIntro_stdpopsim.ipynb)
  - In the Binder associated with the [stdpopsim GitHub repository](https://github.com/popsim-consortium/stdpopsim) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/popsim-consortium/stdpopsim/master?filepath=stdpopsim_example.ipynb)
- Install stdpopsim locally following the [instructions in the documentation](https://stdpopsim.readthedocs.io/en/stable/installation.html)
- Consult the [stdpopsim documentation](https://stdpopsim.readthedocs.io/en/stable/installation.html)

---------
## 10. How to Navigate the [stdpopsim library catalog](https://stdpopsim.readthedocs.io/en/stable/catalog.html)

### The Catalog is organized first by species.

_How many species are there?_

![](images/catalog.png)


### Each species has a set of defining attributes. 

_What are the attributes?_

![](images/species_attributes.png)

### Each species has defined genome parameters.

_What are the genome parameters?_

![](images/genome_params.png)

### Some species have a genetic map.

Genetic maps are stored on AWS and downloaded to cache when used.

![](images/genetic_maps.png)

### Some species have demographic models.

All models are from published models.

_What models are available? Are there any you recognize from the literature?_

![](images/models.png)

### Each model has a description and set of defining attributes. 

_What are the attributes?_

![](images/model_attr.png)

### Each model has a table of defined model parameters from the publication. 

_Can you find where in the original publication the model parameters are given?_

![](images/model_params.png)